In [1]:
# import sys
# sys.path.insert(0, '/w/halld-scshelf2101/lng/WORK/PyAmpTools/external/root/root-6.28.06-gcc9.3.0/lib')

from pyamptools import atiSetup
import numpy as np
import pandas as pd

atiSetup.setup(globals(), verbose=True)

fr = "/w/halld-scshelf2101/malte/final_fullWaveset/nominal_fullWaveset_ReflIndiv_150rnd/010020/etapi_result_samePhaseD.fit"
fr = FitResults(fr)

Welcome to JupyROOT 6.28/06
atiSetup| node called python

------------------------------------------------
atiSetup| MPI is disabled
atiSetup| GPU is disabled
------------------------------------------------


atiSetup| Loading library libIUAmpTools.so ............  ON
atiSetup| Loading library libAmpTools.so ..............  ON
atiSetup| Loading library libAmpPlotter.so ............  ON
atiSetup| Loading library libAmpsDataIO.so ............  ON
atiSetup| Loading library libFSRoot.so ................  OFF
atiSetup| Loading library libAmpsGen.so ...............  OFF


------------------------------------------------
------------------------------------------------

   |        ^                                                      |
   |       / \             Version:  v0.15.3-2-g0753-dirty         |
   |      /---\                                                    |
   |     /     \           GDouble:  8 bytes                       |
   |    /       \ MP           MPI:  NO            

In [2]:
ampAmpPars = dict(fr.ampParMap()) # masses, widths, phase offsets, piecewise bins
ampScales = dict(fr.ampScaleParMap()) # scales between polarized datasets
ampProdPars = dict(fr.ampProdParMap()) # production coefficients

# Quick check to ensure constraints are correct
for k,v in ampProdPars.items():
    x = k.split("::")
    partTag = x[1][-2:]
    conjPart = "Re" if partTag == "Im" else "Im"
    x[1] = x[1][:-2] + conjPart
    assert v == ampProdPars["::".join(x)]
    
# Remove additional waves that are constrained
ampProdPars = {k: v for k,v in ampProdPars.items() if k.split("::")[1][-2:]=="Re"}

# Scale production parameters (S and D), for scaling between polarized datasets
ampScaledProdPars = { key: ampScales[key] * ampProdPars[key] for key in ampProdPars.keys() }

# Ensure amplitude parameters and production parameters are disjoint
assert len(set(ampAmpPars.keys()).intersection(set(ampScaledProdPars.keys()))) == 0

# Merge amplitude and production parameters dictionaries
allPars = {**ampAmpPars, **ampScaledProdPars}

# Determine BW amplitude contributions
massEdges = np.linspace(1.04, 1.72, 17+1)
masses = 0.5 * (massEdges[1:] + massEdges[:-1])
a2mass = allPars['a2mass']
a2width = allPars['a2width']
a2primemass = 1.698
a2primewidth = 0.265
a2_constructor = f"{a2mass} {a2width} 2 0 1".split()
a2prime_constructor = f"{a2primemass} {a2primewidth} 2 0 1".split()
pi0mass = 0.1349
etamass = 0.5478

a2bw = BreitWigner(a2_constructor)
a2primebw = BreitWigner(a2prime_constructor)
a2amps = np.array([a2bw.calcAmplitudeFromMasses(mass, pi0mass, etamass) for mass in masses])
a2primeamps = np.array([a2primebw.calcAmplitudeFromMasses(mass, pi0mass, etamass) for mass in masses])

# Rotate scaled production parameters by PhaseOffset amplitude and multiply BW mass dependence 
all_a2prime = [ key for key in allPars.keys() if "::pD" in key ]
pos_a2prime = [ key for key in all_a2prime if key.endswith("+") ]
neg_a2prime = [ key for key in all_a2prime if key.endswith("-") ]
assert len(pos_a2prime) + len(neg_a2prime) == len(all_a2prime)
all_a2 = [ key for key in allPars.keys() if "::D" in key ]
pos_a2 = [ key for key in all_a2 if key.endswith("+") ]
neg_a2 = [ key for key in all_a2 if key.endswith("-") ]
assert len(pos_a2) + len(neg_a2) == len(all_a2)
for key in pos_a2prime:
    allPars[key] *= np.exp(1j * allPars['a2primePhasePos']) * a2primeamps
for key in neg_a2prime:
    allPars[key] *= np.exp(1j * allPars['a2primePhaseNeg']) * a2primeamps
for key in pos_a2:
    allPars[key] *= np.exp(1j * allPars['a2phasePos']) * a2amps
for key in neg_a2:
    allPars[key] *= np.exp(1j * allPars['a2phaseNeg']) * a2amps

# Load S-wave piecewise mass dependence
Samp = {"S0+": [], "S0-": []}
for bin in range(len(masses)):
    for refl in ["Pos", "Neg"]:
        sign = "+" if refl == "Pos" else "-"
        re = f"pcwsBin_{bin}Re{refl}"
        im = f"pcwsBin_{bin}Im{refl}"
        reAmp = allPars[re] if re in allPars else 0
        imAmp = allPars[im] if im in allPars else 0
        amp = reAmp + 1j * imAmp
        Samp[f"S0{sign}"].append(amp)
Samp = {k: np.array(v) for k,v in Samp.items()}

# Multiply scaled S-wave production parameters by piecewise mass dependence
pos_S = [ key for key in allPars if key.endswith("+") and "::S" in key ]
neg_S = [ key for key in allPars if key.endswith("-") and "::S" in key ] 
for key in pos_S:
    allPars[key] *= Samp["S0+"]
for key in neg_S:
    allPars[key] *= Samp["S0-"]

# Recreate a simpler output dictionary
output = {'mass': masses}
allPars = {k: v for k,v in allPars.items() if all([x not in k for x in ["parScale", "pcws", "phase", "Phase", "a2"]])}
output = {**output, **allPars}
output = pd.DataFrame(output)

# allPars["a2mass"] = a2mass
# allPars["a2width"] = a2width
# allPars["a2primemass"] = a2primemass
# allPars["a2primewidth"] = a2primewidth

output.to_csv("evaluate_amplitude.csv")

In [3]:
print(f"a2mass: {a2mass}")
print(f"a2width: {a2width}")
print(f"a2primemass: {a2primemass}")
print(f"a2primewidth: {a2primewidth}")

a2mass: 1.31746677538279
a2width: 0.133690729750447
a2primemass: 1.698
a2primewidth: 0.265


In [4]:
a2amps

array([ 0.85481265+0.13231771j,  1.03408178+0.21548686j,
        1.26204412+0.35785136j,  1.55435365+0.620081j  ,
        1.89435476+1.14433855j,  2.03617654+2.21287879j,
        0.97592492+3.66520834j, -1.12672   +3.41751913j,
       -1.82264259+2.06168085j, -1.68668355+1.19751789j,
       -1.43207155+0.74920562j, -1.21118348+0.5048143j ,
       -1.03713418+0.36053245j, -0.90094067+0.26915269j,
       -0.79293614+0.20789917j, -0.70577382+0.16493658j,
       -0.63421677+0.13368382j])